# Understanding PyTorch

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/navar 

Mounted at /content/drive
/content/drive/MyDrive/navar


Here we just go through the [Quickstart tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

Importing some data:

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
batch_size = 64 

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # network architecture - MLP
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x) # flatten inputs 
        logits = self.linear_relu_stack(x) # apply MLP and fit/predict
        return logits # return fitted values

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Just setting a loss function and an optimizer:

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Now we still need to define funcions to train and test the model.

> "To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!"


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  # Takes classes and instances we defined above:
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X) # performs predictions
        loss = loss_fn(pred, y) # compute loss

        # Backpropagation
        # Intuition: train in-sample recursively
        # HUNCH(!): for each batch, train model, compute loss
        optimizer.zero_grad() # "take first order conditions" - 1 round of GD?
        loss.backward() # update loss - in-sample (I guess)
        optimizer.step() # update the optimizer to intialize on next turn

        if batch % 100 == 0: # (?)
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Finally, just run the process to train the model. Loss should tend to decrease with each batch and each epoch.

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315787  [    0/60000]
loss: 2.294584  [ 6400/60000]
loss: 2.286385  [12800/60000]
loss: 2.275242  [19200/60000]
loss: 2.263527  [25600/60000]
loss: 2.239128  [32000/60000]
loss: 2.253760  [38400/60000]
loss: 2.219262  [44800/60000]
loss: 2.228657  [51200/60000]
loss: 2.214864  [57600/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 0.034474 

Epoch 2
-------------------------------
loss: 2.216329  [    0/60000]
loss: 2.195725  [ 6400/60000]
loss: 2.152619  [12800/60000]
loss: 2.173091  [19200/60000]
loss: 2.126505  [25600/60000]
loss: 2.070739  [32000/60000]
loss: 2.141737  [38400/60000]
loss: 2.047695  [44800/60000]
loss: 2.089243  [51200/60000]
loss: 2.076654  [57600/60000]
Test Error: 
 Accuracy: 53.4%, Avg loss: 0.031906 

Epoch 3
-------------------------------
loss: 2.072666  [    0/60000]
loss: 2.037302  [ 6400/60000]
loss: 1.947105  [12800/60000]
loss: 2.010952  [19200/60000]
loss: 1.912957  [25600/60000]
loss: 1.832975  [32000/600

## Convolutional 1D

In [19]:
m = nn.Conv1d(16, 33, 3, stride=2) # 16 input channels, 33 output channels, kernel size 3
input = torch.randn(20, 16, 50)
output = m(input)
# We go from 20 matrices of dim (16 x 50) to 20 matrices of size (33 x 25)
output.shape

torch.Size([20, 33, 24])